In [1]:
import networkx as nx
import random
import pulp as p
def generate_undirected_weighted_vertices(num_vertices, num_edges, weight_min= 1, weight_max= 100):
    if(num_edges > ((num_vertices*(num_vertices-1))/2)):
        print("The number of edges exceeds the maximum limit.")
        return None, f"The number of edges ({num_edges}) exceeds the maximum limit ({((num_vertices*(num_vertices-1))/2)})."
    graph = nx.Graph()
    for vertex in range(num_vertices):
        weight = random.randint(weight_min, weight_max)
        graph.add_node(vertex, weight=weight)
    while(len(graph.edges()) < num_edges):
        source = random.randint(0, num_vertices-1)
        destination = random.randint(0, num_vertices-1)
        if(source != destination):
            graph.add_edge(source, destination)   
    return graph, None


def linear_programming(graph):
    Lp_prob = p.LpProblem('Problem', p.LpMinimize)
    variables = [None for _ in graph.nodes]
    for vertex in graph.nodes:
        variables[vertex] = p.LpVariable("x"+str(vertex), lowBound = 0)

    objective_function = ""
    for index, variable in enumerate(variables):
        objective_function +=  graph.nodes[index]['weight'] * variable
    Lp_prob += objective_function

    for variable in variables:
        Lp_prob += variable <= 1

    for edge in graph.edges:
        source = edge[0]
        destination = edge[1]
        condition = "" + variables[source] + variables[destination]  >= 1
        Lp_prob += condition

    status = Lp_prob.solve()
    solution_status = p.LpStatus[status]

    # Printing the final solution
    array_result = []
    for index, variable in enumerate(variables):
        if(p.value(variable) >= 0.5):
            array_result.append(index)
    
    return Lp_prob, p.value(Lp_prob.objective), array_result 

In [2]:
graph = nx.Graph()
weights = [2,0,1,4]
nodes = [0,1,2,3]
edges = [(0,2),(1,0),(3,0),(3,1)]
for index, i in enumerate(nodes):
    graph.add_node(i, weight= weights[index])
graph.add_edges_from(edges)
problem, result_value, vertices = linear_programming(graph)

In [3]:
print("The problem description is : ")
print(problem)
print(f"The results is {vertices} with value of {result_value}.")

The problem description is : 
Problem:
MINIMIZE
2*x0 + 1*x2 + 4*x3 + 0
SUBJECT TO
_C1: x0 <= 1

_C2: x1 <= 1

_C3: x2 <= 1

_C4: x3 <= 1

_C5: x0 + x2 >= 1

_C6: x0 + x1 >= 1

_C7: x0 + x3 >= 1

_C8: x1 + x3 >= 1

VARIABLES
x0 Continuous
x1 Continuous
x2 Continuous
x3 Continuous

The results is [0, 1] with value of 2.0.
